In [1]:
import sys

import os
import argparse
import pandas as pd
import numpy as np
import torch
sys.path.append(r"../")
sys.path.append('/home/wyliu/code/CB-IV')
from utils import log, CausalDataset
from module.SynCBIV import run as run_SynCBIV

os.environ["CUDA_VISIBLE_DEVICES"] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def get_args():
    argparser = argparse.ArgumentParser(description=__doc__)
    # About run setting !!!!
    argparser.add_argument('--seed',default=2021,type=int,help='The random seed')
    argparser.add_argument('--mode',default='vx',type=str,help='The choice of v/x/vx/xx')
    argparser.add_argument('--rewrite_log',default=False,type=bool,help='Whether rewrite log file')
    argparser.add_argument('--use_gpu',default=False,type=bool,help='The use of GPU')
    # About data setting ~~~~
    argparser.add_argument('--num',default=10000,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--num_reps',default=10,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--ate',default=0,type=float,help='The ate of constant')
    argparser.add_argument('--sc',default=1,type=float,help='The sc')
    argparser.add_argument('--sh',default=0,type=float,help='The sh')
    argparser.add_argument('--one',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--depX',default=0.05,type=float,help='Whether generates harder datasets')
    argparser.add_argument('--depU',default=0.05,type=float,help='Whether generates harder datasets')
    argparser.add_argument('--VX',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mV',default=2,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mX',default=4,type=int,help='The dim of Confounding variables X')
    argparser.add_argument('--mU',default=4,type=int,help='The dim of Unobserved confounding variables U')
    argparser.add_argument('--storage_path',default='../Data/',type=str,help='The dir of data storage')
    # Syn
    argparser.add_argument('--syn_alpha',default=0.01,type=float,help='')
    argparser.add_argument('--syn_lambda',default=0.0001,type=float,help='')
    argparser.add_argument('--syn_twoStage',default=True,type=bool,help='')
    # About Debug or Show
    argparser.add_argument('--verbose',default=1,type=int,help='The level of verbose')
    argparser.add_argument('--epoch_show',default=5,type=int,help='The epochs of show time')
    args = argparser.parse_args(args=[])
    return args

args = get_args()

In [3]:
# set path
which_benchmark = 'Syn_'+'_'.join(str(item) for item in [args.sc, args.sh, args.one, args.depX, args.depU,args.VX])
which_dataset = '_'.join(str(item) for item in [args.mV, args.mX, args.mU])
resultDir = args.storage_path + f'/results/{which_benchmark}_{which_dataset}/'
dataDir = f'{args.storage_path}/data/{which_benchmark}/{which_dataset}/'
os.makedirs(os.path.dirname(resultDir), exist_ok=True)
logfile = f'{resultDir}/log.txt'
exp = 0

In [13]:
train_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/train.csv')
# train_df = pd.read_csv(dataDir + f'{exp}/train.csv')
# val_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/val.csv')
# test_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/test.csv')

In [14]:
pd.set_option("display.max_columns",30)
train_df

,u0,u1,u2,u3,x0,x1,x2,x3,v0,v1,z0,p0,s0,m0,m1,t0,y0,f0,c0,c1,c2,c3,c4,c5
0,0.148355,1.002048,1.545932,1.429011,-0.295136,0.710984,-0.304386,1.303714,-1.471120,-0.111800,5.895214,0.997255,0.799299,0.692565,0.980377,1.0,0.961162,0.671763,-1.471120,-0.111800,-0.295136,0.710984,-0.304386,1.303714
1,-0.480872,0.664847,-0.689028,0.434670,-1.159546,0.516137,-1.700485,0.682999,-0.531695,1.523531,-0.328401,0.418630,0.422701,-0.216410,0.788255,0.0,-0.217524,0.798973,-0.531695,1.523531,-1.159546,0.516137,-1.700485,0.682999
2,1.593101,1.186939,0.337385,0.079097,-1.318716,-0.010805,-2.002562,0.800997,0.643073,-0.619137,-0.175905,0.456137,0.108546,0.083179,1.307233,1.0,1.307179,0.086758,0.643073,-0.619137,-1.318716,-0.010805,-2.002562,0.800997
3,0.569034,-0.258703,0.161201,-0.909284,0.266759,-0.943173,-0.851580,-0.998146,-1.347721,0.272128,-3.580073,0.027118,0.154098,-0.370487,0.490717,0.0,-0.361868,0.478071,-1.347721,0.272128,0.266759,-0.943173,-0.851580,-0.998146
4,-0.348029,0.436148,0.965803,0.199437,-0.354222,0.496414,-1.737128,0.005626,1.636188,-1.386524,-1.603814,0.167449,0.120394,-0.041994,0.584181,0.0,-0.047362,0.585022,1.636188,-1.386524,-0.354222,0.496414,-1.737128,0.005626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.269611,-0.292691,0.709657,0.778488,0.704607,-0.177507,-1.302751,-0.755538,-0.157274,1.385244,0.577172,0.640416,0.243768,0.116735,0.700402,0.0,0.104902,0.696519,-0.157274,1.385244,0.704607,-0.177507,-1.302751,-0.755538
9996,-0.928872,0.368577,-2.227690,-0.201655,-0.154100,-0.017975,0.239281,-0.153944,-1.407009,-0.691589,-2.847126,0.054830,0.582732,-0.384547,0.763368,0.0,-0.379801,0.762791,-1.407009,-0.691589,-0.154100,-0.017975,0.239281,-0.153944
9997,0.193575,0.861490,0.262209,0.367476,-0.483780,1.491198,0.355676,0.922763,-0.143283,-0.958113,2.611188,0.931578,0.750860,0.496326,0.552393,1.0,0.550591,0.494340,-0.143283,-0.958113,-0.483780,1.491198,0.355676,0.922763
9998,-1.354232,1.614133,-0.509394,1.070009,-1.452395,-1.422826,0.581264,0.868304,-0.550099,-1.874522,2.860943,0.945882,0.766787,-0.075642,1.383685,1.0,1.397599,-0.076765,-0.550099,-1.874522,-1.452395,-1.422826,0.581264,0.868304
